In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as sp

def main_simulation(system,t_sim,new_r,T_final_exp):
    ''' Finds a proper r value for a known simulation, and then 
    runs a simulation using an unknown time (t_sim) to find the 
    temperature at the later time'''
    if new_r:
        system['T_final_exp'] = T_final_exp
        system['r'] = find_r(system)
        system['t_end'] = t_sim
    results_T, results_T_env = run_simulation(system,change_func)
    print("With a heat transfer coeffient of r =", round(system['r'],5),
          ", the final metal temperature of the metal is ", 
          round(system['T_final'],4), "after ", system['t_end'], " seconds.")
    print("The final liquid temperature is", round(results_T_env.iloc[-1],4))
    results_T.plot(label='Object', xlabel='Time (seconds)',
         ylabel='Temperature (C)',
         title='Temperature of Quenched Object and Liquid',legend=True)
    results_T_env.plot(legend=True,label='Liquid');
    return

def make_system(T_init, mass, cP, T_init_env, vol_env, dens_env, cP_env, r, t_end,dt):
    tmass = mass * cP
    tmass_env = vol_env * dens_env * cP_env
    return dict(T_init=T_init, tmass=tmass,
                  T_init_env=T_init_env, tmass_env=tmass_env,
                  r=r,t_end=t_end,
                  t_0=0, dt=dt)

def make_state(system):
    state = pd.Series(dict(temp=system['T_init'], temp_env=system['T_init_env']))
    return state

def run_simulation(system, change_func):
    
    t_array = np.arange(system['t_0'], system['t_end']+1, system['dt'])
    n = len(t_array)
    state = make_state(system)
    results_T = pd.Series(index=t_array,dtype=object)
    results_T_env = pd.Series(index=t_array,dtype=object)
    results_T.iloc[0] = state.temp
    results_T_env.iloc[0] = state.temp_env
    diff = 0
    
    for i in range(n-1):
        t = t_array[i]
        deltaT, deltaT_env = change_func(t, state, system)
        state.temp = state.temp + deltaT
        state.temp_env = state.temp_env + deltaT_env
        results_T.iloc[i+1] = state.temp
        results_T_env.iloc[i+1] = state.temp_env
        if state.temp - state.temp_env < 5.0 and diff == 0:
            print("At time = ", t, ", the difference between temps is ", state.temp -state.temp_env)
            diff = 1
    system['T_final'] = results_T.iloc[-1]
    system['T_final_env'] = results_T_env.iloc[-1]
    return results_T, results_T_env

def change_func(t, state, system):
    r, dt = system['r'], system['dt']
    deltaT = -r * (state.temp - state.temp_env) * dt
    Q = system['tmass']*deltaT
    deltaT_env = (-Q)/system['tmass_env']
    return deltaT, deltaT_env

def error_func(r, system):
    system['r'] = r
    results = run_simulation(system, change_func)
    return system['T_final'] - system['T_final_exp']

def find_r(system):
    root_obj = sp.root_scalar(error_func, system, bracket=[0, 1.0])
    return root_obj.root


In [ ]:
T_init = 1000.0 # Temp of object, degrees C
mass   = 0.2 # mass of object, kg
cP     = 890.0 # specific heat of object, J/kgC  Iron is 450 J/kgC; alum is 890
T_init_env = 20.0
vol_env = 0.001  # Volume of liquid, m3
dens_env = 1000.0 # Peanut oil is 915.0
cP_env = 2240.0  # Peanut oil is 2240
r = 0.0250
t_end = 120.0 # End of simulation, seconds
dt = 0.1 # time step, seconds
system = make_system(T_init, mass, cP,T_init_env, vol_env,dens_env,cP_env,r,t_end,dt)
results = main_simulation(system, 0,False,0)